In [1]:
from IPython.display import display, clear_output, Javascript, Markdown
import ipywidgets as ipw
import widgets
import utils
import os
import base64
import json
import google.generativeai as genai

Session is no longer valid. Please check if the token is still valid.


In [2]:
CONFIG = utils.read_json("config.json")
CONFIG_ELN = utils.get_aiidalab_eln_config()
# CONFIG_ELN = utils.read_json("eln_config.json")
OPENBIS_SESSION, SESSION_DATA = utils.connect_openbis(CONFIG_ELN["url"], CONFIG_ELN["token"])

# Dropdown box for getting openBIS objects
publication_selector = widgets.PublicationSelectionWidget()
publication_selector.load_dropdown_box()

prompt_textarea = utils.Textarea(
    description = "Prompt", 
    layout = ipw.Layout(width = '980px', height = '300px'),
    style = {"description_width": "110px"}
)

model_answer_textarea = utils.Textarea(
    description = "Answer", 
    layout = ipw.Layout(width = '980px', height = '300px'),
    style = {"description_width": "110px"}
)

enter_button = utils.Button(
    description = '', disabled = False, button_style = '', 
    tooltip = 'Enter', icon = 'arrow-right', layout = ipw.Layout(width = '100px', height = '50px')
)

load_model_button = utils.Button(
    description = '', disabled = False, button_style = '', 
    tooltip = 'Load chatbot', icon = 'arrow-down', layout = ipw.Layout(width = '100px', height = '50px')
)

download_button = utils.Button(
    description = '', disabled = False, button_style = '', 
    tooltip = 'Download', icon = 'download', layout = ipw.Layout(width = '100px', height = '50px')
)

quit_button = utils.Button(
    description = '', disabled = False, button_style = '', 
    tooltip = 'Main menu', icon = 'home', layout = ipw.Layout(width = '100px', height = '50px')
)

load_download_buttons_hbox = ipw.HBox([load_model_button, download_button])

increase_buttons_size = utils.HTML(data = ''.join(CONFIG["save_home_buttons_settings"]))

# Google Gemini 1.5 Flash
google_api_key = utils.read_json("/home/jovyan/gemini_api.json")
genai.configure(api_key=google_api_key["api_key"])

model_name = "gemini-1.5-flash"
system_instruction = "You are a materials science expert working with nanotech materials."
model = genai.GenerativeModel(
    model_name = model_name, 
    system_instruction = system_instruction
)

# Start the chatbot
chat = model.start_chat()

Session is no longer valid. Please check if the token is still valid.


AttributeError: 'NoneType' object has no attribute 'get_objects'

In [ ]:
def close_notebook(b):
    display(utils.Javascript(data = 'window.location.replace("home.ipynb")'))

def ask_chatbot(change):
    model_answer_textarea.value = ""
    prompt = prompt_textarea.value
    response = chat.send_message(prompt, stream = True)
    for chunk in response:
        model_answer_textarea.value = model_answer_textarea.value + chunk.text
    
def get_openbis_object_metadata():
    if publication_selector.dropdown.value == -1:
        return None
    else:
        openbis_selected_object_permid = publication_selector.dropdown.value
        openbis_selected_object = OPENBIS_SESSION.get_sample(openbis_selected_object_permid)

        # Collect all the information about the object from openBIS
        parent_child_relationships = openbis_selected_object.props.all()
        parent_child_relationships["perm_id"] = openbis_selected_object_permid
        parent_child_relationships["type"] = str(openbis_selected_object.type)
        parent_child_relationships["registrationDate"] = openbis_selected_object.registrationDate
        openbis_objects_history = {}
        selected_object_schema, _ = utils.get_parent_child_relationships_nested(
            OPENBIS_SESSION, openbis_selected_object, parent_child_relationships, openbis_objects_history
        )

    return selected_object_schema

def load_chatbot(b):
    data_dict = get_openbis_object_metadata()
    if data_dict:
        json_data = json.dumps(data_dict)
        history = [
            {"role": "user", "parts": [{"text": f"Hello, this string contains a JSON object database: {json_data} \n. Be ready to answer questions about this."}]},
            {"role": "model", "parts": [{"text": "Hi, ok feel free to ask me questions about that."}]}
        ]
        chat.history = history
        display(utils.Javascript(data = "alert('Chatbot is ready!')"))

def download_metadata(b):
    data_dict = get_openbis_object_metadata()
    if data_dict:
        # Export to JSON
        json_data = json.dumps(data_dict, indent=4).encode('utf-8')
        json_data = base64.b64encode(json_data).decode('utf-8')
        
        display(Javascript(f"""
            var a = document.createElement('a');
            a.href = 'data:text/json;base64,{json_data}';
            a.download = 'data.json';
            document.body.appendChild(a);
            a.click();
            document.body.removeChild(a);
        """))

# Export publication metadata

## Select publication

In [ ]:
display(increase_buttons_size)
display(publication_selector)
display(load_download_buttons_hbox)
display(prompt_textarea)
display(enter_button)
display(model_answer_textarea)
display(quit_button)
load_model_button.on_click(load_chatbot)
download_button.on_click(download_metadata)
enter_button.on_click(ask_chatbot)
quit_button.on_click(close_notebook)